# Masinsko ucenje - projekat

Dataset: https://archive.ics.uci.edu/ml/datasets/Diabetes+130-US+hospitals+for+years+1999-2008
 
Klasifikacija - Predviđanje ponovne hospitalizacije pacijenta sa dijabetesom na osnovu podataka sa inicijalne hospitalizacije

1.   Priprema podataka

In [ ]:
import pandas as pd
import sklearn as scikit
import numpy as np
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer
import copy
import matplotlib.pyplot as plt
import category_encoders as ce
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
import shap

In [ ]:
path = "../dataset/diabetic_data.csv"
dataframe = pd.read_csv(path, low_memory=False, na_values=["?", "Unknown/Invalid"])
pd.set_option("display.max_columns", None)
dataframe.head(3)

In [ ]:
total_rows = dataframe.shape[0]
dataframe.shape

In [ ]:
dataframe["readmitted"].value_counts()

In [ ]:
missing = dataframe.isna()
missing_count = missing.sum()
treshold = 0.2
to_drop = []
for i in range(0, missing_count.count()):
    if missing_count[i] / total_rows > treshold:
        to_drop.append(i)
for i in to_drop:
    print(i, dataframe.columns[i])
dataframe = dataframe.drop(dataframe.columns[to_drop], axis=1)
dataframe.shape


In [ ]:
dataframe.dropna(inplace=True)
dataframe.drop(
    dataframe[
        [
            "encounter_id",
            "patient_nbr",
            "examide",
            "citoglipton",
            "metformin-rosiglitazone",
        ]
    ],
    axis=1,
    inplace=True,
)


In [ ]:
print("Number Of Rows In The Original DataFrame:", len(dataframe))
print("Number Of Rows After Deduping:", len(dataframe.drop_duplicates()))


In [ ]:
dataframe.info()

Diagnose mapping http://icd9cm.chrisendres.com/index.php?action=contents

In [ ]:
def diag_mapper(value: str) -> int:
    try:
        int_value = int(float(value))
        if int_value <= 139:
            return 'Infectious and Parasitic'
        elif int_value <= 239:
            return 'Neoplasms'
        elif int_value <= 279:
            return 'Metabolic'
        elif int_value <= 289:
            return 'Blood'
        elif int_value <= 319:
            return 'Mental'
        elif int_value <= 389:
            return 'Nervous System'
        elif int_value <= 459:
            return 'Circulatory System'
        elif int_value <= 519:
            return 'Respiratory System'
        elif int_value <= 579:
            return 'Digestive System'
        elif int_value <= 629:
            return 'Genitourinary System'
        elif int_value <= 679:
            return 'Pregnancy'
        elif int_value <= 709:
            return 'Skin'
        elif int_value <= 739:
            return 'Musculoskeletal'
        elif int_value <= 759:
            return 'Congenital'
        elif int_value <= 779:
            return 'Perinatal Period'
        elif int_value <= 799:
            return 'Ill Defined'
        else:
            return 'Injury'
    except ValueError:
        if value[0] == 'V':
            return 'Status'
        else:  # E
            return 'Cause'


dataframe['diag_1_class'] = dataframe['diag_1'].apply(lambda x: diag_mapper(x))
dataframe['diag_2_class'] = dataframe['diag_2'].apply(lambda x: diag_mapper(x))
dataframe['diag_3_class'] = dataframe['diag_3'].apply(lambda x: diag_mapper(x))


In [ ]:
age_scale_mapper = {
    '[0-10)': 0,
    '[10-20)': 1,
    '[20-30)': 2,
    '[30-40)': 3,
    '[40-50)': 4,
    '[50-60)': 5,
    '[60-70)': 6,
    '[70-80)': 7,
    '[80-90)': 8,
    '[90-100)': 9,
}
glu_scale_mapper = {
    'None': 70,
    'Norm': 70,
    '>200': 250,
    '>300': 350
}
a1_scale_mapper = {
    'None': 4,
    'Norm': 4,
    '>7': 7,
    '>8': 9,
}
level_scale_mapper = {
    'No': 0,
    'Steady': 2,
    'Up': 3,
    'Down': 1
}
class_mapper = {
    'NO': 0,
    '<30': 1,
    '>30': 2
}
race_mapper = {
    "Other": 0,
    "Caucasian": 1,
    "AfricanAmerican": 2,
    "Hispanic": 3,
    "Asian": 4,
}
diagnum_mapper = {
    'Infectious and Parasitic': 0,
    'Neoplasms': 1,
    'Metabolic': 2,
    'Blood': 3,
    'Mental': 4,
    'Nervous System': 5,
    'Circulatory System': 6,
    'Respiratory System': 7,
    'Digestive System': 8,
    'Genitourinary System': 9,
    'Pregnancy': 10,
    'Skin': 11,
    'Musculoskeletal': 12,
    'Congenital': 13,
    'Perinatal Period': 14,
    'Ill Defined': 15,
    'Injury': 16,
    'Status': 17,
    'Cause': 18,
}

one_hot = LabelBinarizer()

dataframe['admission_type_id'] = dataframe['admission_type_id'].astype(
    'category')
dataframe['discharge_disposition_id'] = dataframe['discharge_disposition_id'].astype(
    'category')
dataframe['admission_source_id'] = dataframe['admission_source_id'].astype(
    'category')

dataframe['age'] = dataframe['age'].replace(age_scale_mapper)
dataframe['gender'] = one_hot.fit_transform(dataframe['gender'])
dataframe['max_glu_serum'] = dataframe['max_glu_serum'].replace(
    glu_scale_mapper)
dataframe['A1Cresult'] = dataframe['A1Cresult'].replace(a1_scale_mapper)
dataframe['metformin'] = dataframe['metformin'].replace(level_scale_mapper)
dataframe['repaglinide'] = dataframe['repaglinide'].replace(level_scale_mapper)
dataframe['nateglinide'] = dataframe['nateglinide'].replace(level_scale_mapper)
dataframe['chlorpropamide'] = dataframe['chlorpropamide'].replace(
    level_scale_mapper)
dataframe['glimepiride'] = dataframe['glimepiride'].replace(level_scale_mapper)
dataframe['acetohexamide'] = dataframe['acetohexamide'].replace(
    level_scale_mapper)
dataframe['glipizide'] = dataframe['glipizide'].replace(level_scale_mapper)
dataframe['glyburide'] = dataframe['glyburide'].replace(level_scale_mapper)
dataframe['tolbutamide'] = dataframe['tolbutamide'].replace(level_scale_mapper)
dataframe['pioglitazone'] = dataframe['pioglitazone'].replace(
    level_scale_mapper)
dataframe['rosiglitazone'] = dataframe['rosiglitazone'].replace(
    level_scale_mapper)
dataframe['acarbose'] = dataframe['acarbose'].replace(level_scale_mapper)
dataframe['miglitol'] = dataframe['miglitol'].replace(level_scale_mapper)
dataframe['troglitazone'] = dataframe['troglitazone'].replace(
    level_scale_mapper)
dataframe['tolazamide'] = dataframe['tolazamide'].replace(level_scale_mapper)
dataframe['insulin'] = dataframe['insulin'].replace(level_scale_mapper)
dataframe['glyburide-metformin'] = dataframe['glyburide-metformin'].replace(
    level_scale_mapper)
dataframe['glipizide-metformin'] = dataframe['glipizide-metformin'].replace(
    level_scale_mapper)
dataframe['glimepiride-pioglitazone'] = dataframe['glimepiride-pioglitazone'].replace(
    level_scale_mapper)
dataframe['metformin-pioglitazone'] = dataframe['metformin-pioglitazone'].replace(
    level_scale_mapper)
dataframe['change'] = one_hot.fit_transform(dataframe['change'])
dataframe['diabetesMed'] = one_hot.fit_transform(dataframe['diabetesMed'])
dataframe['readmitted'] = dataframe['readmitted'].replace(class_mapper)


In [ ]:
dataframe_onehot = dataframe.copy(deep=True)
dataframe.info()

In [ ]:
def df_one_hot(df,columns):
    tmp=pd.DataFrame()
    for col in columns:
        res=one_hot.fit_transform(df[col])
        for val in range(0,len(res[0])):
            new_col = res[:,val]
            tmp_col = pd.DataFrame({col +'('+ str(one_hot.classes_[val])+')' : new_col})
            tmp = pd.concat((tmp,tmp_col),axis=1)
    return pd.concat((df.reset_index(drop=True),tmp.reset_index(drop=True)),axis=1, join='inner')

dataframe_onehot=df_one_hot(dataframe_onehot,['diag_1_class','diag_2_class','diag_3_class','admission_type_id','discharge_disposition_id','admission_source_id','race'])
dataframe_onehot.drop(['diag_1_class','diag_2_class','diag_3_class','admission_type_id','discharge_disposition_id','admission_source_id','race'], axis=1, inplace=True)
dataframe_onehot.shape

In [ ]:
dataframe['diag_1_class'] = dataframe['diag_1'].replace(diagnum_mapper)
dataframe['diag_2_class'] = dataframe['diag_2'].replace(diagnum_mapper)
dataframe['diag_3_class'] = dataframe['diag_3'].replace(diagnum_mapper)
dataframe['race'] = dataframe['race'].replace(race_mapper)

In [ ]:
print(dataframe.shape)
dataframe.head()

In [ ]:
print(dataframe_onehot.shape)
dataframe_onehot.head()

2.   Deskriptivna analiza

In [ ]:
# names for categories
x = dataframe['readmitted'].unique()
# quantities for each category
y = dataframe['readmitted'].value_counts()
# create bar chart
plt.bar(x, y)
# show line plot
plt.show()


In [ ]:
def feature_importance(df):
    target = 'readmitted'
    y = df[target]
    X = df.drop(columns=[target])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=1066)
    encoder = ce.LeaveOneOutEncoder(return_df=True)
    X_train_loo = encoder.fit_transform(X_train, y_train)
    X_test_loo = encoder.transform(X_test)
    model = GradientBoostingRegressor(learning_rate=0.05, max_depth=5, n_estimators=500, min_samples_split=5, n_iter_no_change=10)
    model.fit(X_train_loo, y_train)

    rmse = np.sqrt(mean_squared_error(y_test, model.predict(X_test_loo)))
    feature_importance = model.feature_importances_
    sorted_idx = np.argsort(feature_importance)
    fig = plt.figure(figsize=(20,30))
    plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align='center')
    plt.yticks(range(len(sorted_idx)), np.array(X_test.columns)[sorted_idx])
    plt.title('Feature Importance')

    perm_importance = permutation_importance(model, X_test_loo, y_test, n_repeats=10, random_state=1066)
    sorted_idx = perm_importance.importances_mean.argsort()
    fig = plt.figure(figsize=(20,30))
    plt.barh(range(len(sorted_idx)), perm_importance.importances_mean[sorted_idx], align='center')
    plt.yticks(range(len(sorted_idx)), np.array(X_test.columns)[sorted_idx])
    plt.title('Permutation Importance')

    explainer = shap.Explainer(model)
    shap_values = explainer(X_test_loo)
    shap_importance = shap_values.abs.mean(0).values
    sorted_idx = shap_importance.argsort()
    fig = plt.figure(figsize=(20,30))
    plt.barh(range(len(sorted_idx)), shap_importance[sorted_idx], align='center')
    plt.yticks(range(len(sorted_idx)), np.array(X_test.columns)[sorted_idx])
    plt.title('SHAP Importance')

feature_importance(dataframe_onehot)